In [1]:
from azure.cosmos import CosmosClient, exceptions, PartitionKey
import os
import json

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [2]:
# Create a database

URL = os.environ['AZURE_COSMOSDB_ENDPOINT']
KEY = os.environ['AZURE_COSMOSDB_KEY']
client = CosmosClient(URL, credential=KEY)
DATABASE_NAME = 'coffeeDatabase'
try:
    database = client.create_database(DATABASE_NAME)
except exceptions.CosmosResourceExistsError:
    database = client.get_database_client(DATABASE_NAME)
database

<DatabaseProxy [dbs/coffeeDatabase]>

In [3]:
properties = database.read()
print(json.dumps(properties, indent=4))

{
    "id": "coffeeDatabase",
    "_rid": "uBp0AA==",
    "_self": "dbs/uBp0AA==/",
    "_etag": "\"00005408-0000-0500-0000-656966f40000\"",
    "_colls": "colls/",
    "_users": "users/",
    "_ts": 1701406452
}


In [4]:
# Create a container

CONTAINER_NAME = 'menuContainer'

try:
    container = database.create_container(id=CONTAINER_NAME, partition_key=PartitionKey(path="/item_id"))
except exceptions.CosmosResourceExistsError:
    container = database.get_container_client(CONTAINER_NAME)
except exceptions.CosmosHttpResponseError:
    raise

container

<ContainerProxy [dbs/coffeeDatabase/colls/menuContainer]>

In [5]:
# Upload data to Cosmos DB container
with open('../data/contoso.json') as f:  
    documents = json.load(f)  

for document in documents:  
    container.upsert_item(body=document)

In [6]:
# Query the data  
query = "SELECT * FROM c"  
items = list(container.query_items(  
    query=query,  
    enable_cross_partition_query=True  
))  
  
# Convert the data to JSON  
json_data = json.dumps(items, indent=4)

items[0]
 

{'id': 'c001',
 'item': 'Espresso',
 'price': 2.5,
 'description': 'Rich, full-bodied coffee, with a compact crema and strong aroma.',
 'category': 'Coffees',
 '_rid': 'uBp0AIaQaQIBAAAAAAAAAA==',
 '_self': 'dbs/uBp0AA==/colls/uBp0AIaQaQI=/docs/uBp0AIaQaQIBAAAAAAAAAA==/',
 '_etag': '"11003bd2-0000-0500-0000-658dc87e0000"',
 '_attachments': 'attachments/',
 '_ts': 1703790718}